In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import numpy as np

import ray
import resnet.models as models
import random,time
from time import sleep
import copy 

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import argparse
import numpy as np
import os
import shutil

if ray.is_initialized():
    ray.shutdown()
ray.init(memory=10000000000,object_store_memory=6000000000)


2019-11-01 14:00:31,336	WARNING worker.py:1426 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2019-11-01 14:00:31,345	INFO resource_spec.py:205 -- Starting Ray with 9.28 GiB memory available for workers and up to 5.59 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).


{'node_ip_address': '10.21.5.177',
 'redis_address': '10.21.5.177:63974',
 'object_store_address': '/tmp/ray/session_2019-11-01_14-00-31_339910_29637/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-11-01_14-00-31_339910_29637/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2019-11-01_14-00-31_339910_29637'}

In [2]:
arch = "resnet"
depth = 56
cuda = torch.cuda.is_available()
cuda = False
seed = 1
save = "./logs"
dataset = "cifar10"
batch_size = 64
test_batch_size = 65
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

lr = 0.1
momentum=0.9
weight_decay=1e-4
log_interval=100
start_epoch = 0
epochs=160

In [3]:
if not os.path.exists(save):
    os.makedirs(save)

In [4]:
if dataset == "cifar10":
    train_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10('./data.cifar10', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.Pad(4),
                           transforms.RandomCrop(32),
                           transforms.RandomHorizontalFlip(),
                           transforms.ToTensor(),
                           transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
                       ])),
        batch_size=batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10('./data.cifar10', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
                       ])),
        batch_size=test_batch_size, shuffle=True, **kwargs)

Files already downloaded and verified


In [5]:
@ray.remote(num_gpus=0.3)
# @ray.remote
class ParameterServer():
    def __init__(self,lr,num_workers,stalness_limit,test_loader):
        self.lr = lr
        self.model = models.__dict__[arch](dataset=dataset,depth=depth)
        self.stalness_table = [0] * num_workers
        self.stalness_limit = stalness_limit 
        self.global_step = 0
        self.eva_model = models.__dict__[arch](dataset=dataset,depth=depth)
        self.optimizer = optim.SGD(self.model.parameters(),
                          lr=lr,
                          momentum=momentum,
                          weight_decay=weight_decay)
        self.test_loader = test_loader
        if cuda:
            self.model.cuda()
            self.eva_model.cuda()
    def apply_gradients(self, gradients, wk_idx):
        for idx, p in enumerate(self.model.parameters()):
            p.data -= self.lr * gradients[idx]
        self.stalness_table[wk_idx] += 1
        self.global_step += 1
        if self.global_step % 100 == 0:
            print("global_step: ",self.global_step," and prepare evaluate")
            self.evaluate()
        
    def pull_weights(self):
        return self.model.state_dict()
    
    def check_stalness(self,wk_idx):
        min_iter = min(self.stalness_table)
        return self.stalness_table[wk_idx] - min_iter < self.stalness_limit
        
    def get_stalness(self):
        return min(self.stalness_table)
    
    def evaluate(self):
        print("going to evaluate")
        self.eva_model.eval()
        test_loss = 0.
        correct = 0.
        cur_wei = self.pull_weights()
        print("pulled weights")
        self.eva_model.load_state_dict(cur_wei)
        print("loaded weights")
        print("length of the test_loader dataset is : ",len(test_loader.dataset))
        batch = iter(test_loader)
        data,target = next(batch)
        if cuda: 
            data,target = data.cuda(),target.cuda()
        data,target = Variable(data,volatile=True),Variable(target)
        output = self.eva_model(data)
        test_loss = F.cross_entropy(output,target,size_average=True)
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct = pred.eq(target.data.view_as(pred)).cpu().sum()

        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.1f}%)\n'.format(
            test_loss, 
            correct, 
            len(data),
            100. * correct / len(data)))


        
@ray.remote(num_gpus=0.3)
# @ray.remote
def worker_task(ps,worker_index,stale_limit, train_loader,lr,momentum,weight_decay,batch_size=50):
    # Initialize the model.
    model = models.__dict__[arch](dataset=dataset,depth=depth)
    local_step = 0
    optimizer = optim.SGD(model.parameters(),
                          lr=lr,
                          momentum=momentum,
                          weight_decay=weight_decay)
    if cuda:
        model.cuda()
    for batch_idx,(data,target) in enumerate(train_loader):
        if cuda:
            data,target = data.cuda(),target.cuda()
        while(local_step - ray.get(ps.get_stalness.remote()) > stale_limit):
            print(worker_index," works too fast")
            sleep(1)
        # Get the current weights from the parameter server.
        init_wei = ray.get(ps.pull_weights.remote())
        model.load_state_dict(init_wei)
        
        # Compute an update and push it to the parameter server.        
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        loss = F.cross_entropy(model(data),target)
        loss.backward()
        grad = [p.grad for p in model.parameters()]
        local_step += 1
        ps.apply_gradients.remote(grad,worker_index)
        optimizer.step()
        print(worker_index,"has finished update")


In [6]:
num_worker = 2
stalness_table = [0] * num_worker
stalness_limit = 4

ps = ParameterServer.remote(0.1,num_worker,stalness_limit,test_loader)


2019-11-01 14:00:43,197	WARNING worker.py:1779 -- Warning: The actor ParameterServer has size 30745381 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.


In [7]:
worker_tasks = [worker_task.remote(ps,i,stalness_limit,train_loader,lr,momentum,weight_decay) 
                for i in range(num_worker)]

(pid=29674) 0 has finished update
(pid=29674) 0 has finished update
(pid=29674) 0 has finished update
(pid=29674) 0 has finished update
(pid=29674) 0 has finished update
(pid=29674) 0  works too fast
(pid=29674) 0  works too fast
(pid=29672) 1 has finished update
(pid=29672) 1 has finished update
(pid=29674) 0 has finished update
(pid=29672) 1 has finished update
(pid=29674) 0 has finished update
(pid=29672) 1 has finished update
(pid=29674) 0 has finished update
(pid=29672) 1 has finished update
(pid=29674) 0 has finished update
(pid=29672) 1 has finished update
(pid=29674) 0 has finished update
(pid=29672) 1 has finished update
(pid=29674) 0 has finished update
(pid=29672) 1 has finished update
(pid=29674) 0 has finished update
(pid=29672) 1 has finished update
(pid=29674) 0 has finished update
(pid=29672) 1 has finished update
(pid=29674) 0 has finished update
(pid=29672) 1 has finished update
(pid=29674) 0 has finished update
(pid=29672) 1 has finished update
(pid=29674) 0 has fini